In [1]:
# importing grpc via python
import grpc

import swh.graph.grpc.swhgraph_pb2 as swhgraph
import swh.graph.grpc.swhgraph_pb2_grpc as swhgraph_grpc
from google.protobuf.field_mask_pb2 import FieldMask

[Example Dataset](https://docs.softwareheritage.org/devel/swh-graph/example-dataset.html)

<img src="https://docs.softwareheritage.org/_images/example-dataset.svg" width="500"/>

In [2]:
# address of the localhost if GRPC induced from java.graph.jar 
GRAPH_GRPC_SERVER = "localhost:50091"

In [3]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.Stats(swhgraph.StatsRequest())
    print(response)
    print("Compression ratio:", response.compression_ratio * 100, "%")

num_nodes: 24
num_edges: 28
compression_ratio: 1.406
bits_per_node: 9.25
bits_per_edge: 7.929
avg_locality: 3.143
indegree_max: 4
indegree_avg: 1.1666666666666667
outdegree_max: 3
outdegree_avg: 1.1666666666666667
export_started_at: 1669888200
export_ended_at: 1669899600

Compression ratio: 140.6 %


In [4]:
# to induce grpc channel
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    pass  # <insert snippet here> # swhid and response from grpc request

### getting the content node 

In [7]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    swhid = "swh:1:cnt:0000000000000000000000000000000000000015"
    response = stub.GetNode(swhgraph.GetNodeRequest(swhid=swhid))
    print(response)

swhid: "swh:1:cnt:0000000000000000000000000000000000000015"
cnt {
  length: 404
  is_skipped: true
}



### getting the revision node

In [8]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    swhid = "swh:1:rev:0000000000000000000000000000000000000018"
    response = stub.GetNode(swhgraph.GetNodeRequest(swhid=swhid))
    print(response)

swhid: "swh:1:rev:0000000000000000000000000000000000000018"
rev {
  author: 1
  author_date: 1111177770
  author_date_offset: 0
  committer: 0
  committer_date: 1111177770
  committer_date_offset: 0
  message: "Refactor codebase"
}



### getting the release node

In [20]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    swhid = "swh:1:rel:0000000000000000000000000000000000000021"
    response = stub.GetNode(swhgraph.GetNodeRequest(swhid=swhid))
    print(response)

swhid: "swh:1:rel:0000000000000000000000000000000000000021"
rel {
  name: "v2.0-anonymous"
  message: "Version 2.0 but with no author"
}



### getting the snapshot node 

In [17]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    swhid = "swh:1:snp:0000000000000000000000000000000000000022"
    response = stub.GetNode(swhgraph.GetNodeRequest(swhid=swhid))
    print(response)
    #  this is (redundant because it only brings SWHID)

swhid: "swh:1:snp:0000000000000000000000000000000000000022"



### getting the origin

In [21]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    swhid = "swh:1:ori:83404f995118bd25774f4ac14422a8f175e7a054"
    response = stub.GetNode(swhgraph.GetNodeRequest(swhid=swhid))
    print(response)
    #  this is (redundant because it only brings SWHID)

swhid: "swh:1:ori:83404f995118bd25774f4ac14422a8f175e7a054"
ori {
  url: "https://example.com/swh/graph"
}



### wrong swhid error simulation

In [23]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    swhid = "swh:1:ori:ffffffffffffffffffffffffffffffffffffffff"
    response = stub.GetNode(swhgraph.GetNodeRequest(swhid=swhid))
    print(response)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INVALID_ARGUMENT
	details = "Unknown SWHID: swh:1:ori:ffffffffffffffffffffffffffffffffffffffff"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"Unknown SWHID: swh:1:ori:ffffffffffffffffffffffffffffffffffffffff", grpc_status:3, created_time:"2024-07-23T16:08:41.904975009+02:00"}"
>

### the code below just to filter out SWHID

In [28]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.GetNode(swhgraph.GetNodeRequest(
    swhid="swh:1:rev:0000000000000000000000000000000000000018",
    mask=FieldMask(paths=["swhid"])
))
print(response)

swhid: "swh:1:rev:0000000000000000000000000000000000000018"
rev {
}



### filtering information further

In [29]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.GetNode(swhgraph.GetNodeRequest(
    swhid="swh:1:rev:0000000000000000000000000000000000000018",
    mask=FieldMask(paths=["swhid", "rev.message", "rev.author"])
))
print(response)    # filtering revision swhid message and author

swhid: "swh:1:rev:0000000000000000000000000000000000000018"
rev {
  author: 1
  message: "Refactor codebase"
}



In [13]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    swhid="swh:1:rev:0000000000000000000000000000000000000018"
    response = stub.GetNode(swhgraph.GetNodeRequest(swhid=swhid,
    mask=FieldMask(paths=["swhid", "rev.message", "rev.author"])))
    print(response)  

swhid: "swh:1:rev:0000000000000000000000000000000000000018"
rev {
  author: 1
  message: "Refactor codebase"
}



In [33]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.GetNode(swhgraph.GetNodeRequest(
    swhid="swh:1:rev:0000000000000000000000000000000000000018",
    mask=FieldMask(paths=["swhid", "rev.author_date", "rev.author"]) # it is not necessary to contain swhid but if it does not contain how can I know which revision is it?
))
print(response)   # prints the revision belonging to author and date

swhid: "swh:1:rev:0000000000000000000000000000000000000018"
rev {
  author: 1
  author_date: 1111177770
}



### stats on the graph

In [39]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.Stats(swhgraph.StatsRequest())
    print(response)

num_nodes: 24
num_edges: 28
compression_ratio: 1.406
bits_per_node: 9.25
bits_per_edge: 7.929
avg_locality: 3.143
indegree_max: 4
indegree_avg: 1.1666666666666667
outdegree_max: 3
outdegree_avg: 1.1666666666666667
export_started_at: 1669888200
export_ended_at: 1669899600



### Graph traversals --so it is passing through certain nodes-

In [63]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    swhid = "swh:1:dir:0000000000000000000000000000000000000006"
    response = stub.Traverse(swhgraph.TraversalRequest(src=[swhid]))
    for item in response:
        print(item)

swhid: "swh:1:dir:0000000000000000000000000000000000000006"
successor {
  swhid: "swh:1:cnt:0000000000000000000000000000000000000005"
  label {
    name: "parser.c"
    permission: 33188
  }
}
successor {
  swhid: "swh:1:cnt:0000000000000000000000000000000000000004"
  label {
    name: "README.md"
    permission: 33188
  }
}
num_successors: 2

swhid: "swh:1:cnt:0000000000000000000000000000000000000005"
cnt {
  length: 1337
  is_skipped: false
}

swhid: "swh:1:cnt:0000000000000000000000000000000000000004"
cnt {
  length: 404
  is_skipped: false
}



In [117]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    src = "swh:1:dir:0000000000000000000000000000000000000008"
    response = stub.Traverse(swhgraph.TraversalRequest(src=[src],
    mask=FieldMask(paths=["swhid", "successor"])))
    for item in response:
        print(f'swhid: "{item.swhid}"')
        print(f'successor: "{item.successor}"')

swhid: "swh:1:dir:0000000000000000000000000000000000000008"
successor: "[swhid: "swh:1:dir:0000000000000000000000000000000000000006"
label {
  name: "tests"
  permission: 33261
}
, swhid: "swh:1:cnt:0000000000000000000000000000000000000001"
label {
  name: "README.md"
  permission: 33188
}
, swhid: "swh:1:cnt:0000000000000000000000000000000000000007"
label {
  name: "parser.c"
  permission: 33188
}
]"
swhid: "swh:1:dir:0000000000000000000000000000000000000006"
successor: "[swhid: "swh:1:cnt:0000000000000000000000000000000000000005"
label {
  name: "parser.c"
  permission: 33188
}
, swhid: "swh:1:cnt:0000000000000000000000000000000000000004"
label {
  name: "README.md"
  permission: 33188
}
]"
swhid: "swh:1:cnt:0000000000000000000000000000000000000001"
successor: "[]"
swhid: "swh:1:cnt:0000000000000000000000000000000000000007"
successor: "[]"
swhid: "swh:1:cnt:0000000000000000000000000000000000000005"
successor: "[]"
swhid: "swh:1:cnt:0000000000000000000000000000000000000004"
successor:

In [83]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    src = "swh:1:dir:0000000000000000000000000000000000000008"
    response = stub.Traverse(swhgraph.TraversalRequest(src=[src]))
    for item in response:
        print(item)

swhid: "swh:1:dir:0000000000000000000000000000000000000008"
successor {
  swhid: "swh:1:dir:0000000000000000000000000000000000000006"
  label {
    name: "tests"
    permission: 33261
  }
}
successor {
  swhid: "swh:1:cnt:0000000000000000000000000000000000000001"
  label {
    name: "README.md"
    permission: 33188
  }
}
successor {
  swhid: "swh:1:cnt:0000000000000000000000000000000000000007"
  label {
    name: "parser.c"
    permission: 33188
  }
}
num_successors: 3

swhid: "swh:1:dir:0000000000000000000000000000000000000006"
successor {
  swhid: "swh:1:cnt:0000000000000000000000000000000000000005"
  label {
    name: "parser.c"
    permission: 33188
  }
}
successor {
  swhid: "swh:1:cnt:0000000000000000000000000000000000000004"
  label {
    name: "README.md"
    permission: 33188
  }
}
num_successors: 2

swhid: "swh:1:cnt:0000000000000000000000000000000000000001"
cnt {
  length: 42
  is_skipped: false
}

swhid: "swh:1:cnt:0000000000000000000000000000000000000007"
cnt {
  length: 

### filtering the SWHIDs of the connecting content nodes to directory

In [10]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    src = "swh:1:dir:0000000000000000000000000000000000000006"
    response = stub.Traverse(swhgraph.TraversalRequest(src=[src],
    mask=FieldMask(paths=["swhid"])))
    for item in response:
        print(f'swhid: "{item.swhid}"')

swhid: "swh:1:dir:0000000000000000000000000000000000000006"
swhid: "swh:1:cnt:0000000000000000000000000000000000000005"
swhid: "swh:1:cnt:0000000000000000000000000000000000000004"


In [12]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    src = "swh:1:dir:0000000000000000000000000000000000000017"
    response = stub.Traverse(swhgraph.TraversalRequest(src=[src],
    mask=FieldMask(paths=["swhid"])))
    for item in response:
        print(f'swhid: "{item.swhid}"')

swhid: "swh:1:dir:0000000000000000000000000000000000000017"
swhid: "swh:1:dir:0000000000000000000000000000000000000016"
swhid: "swh:1:cnt:0000000000000000000000000000000000000014"
swhid: "swh:1:cnt:0000000000000000000000000000000000000015"


### direction of the graph

In [15]:
# starting from directory 6 moving up...
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.Traverse(swhgraph.TraversalRequest(
    src=["swh:1:dir:0000000000000000000000000000000000000006"],
    direction=swhgraph.GraphDirection.BACKWARD,
    mask=FieldMask(paths=["swhid"])))
    for item in response:
        print(f'swhid: "{item.swhid}"')

swhid: "swh:1:dir:0000000000000000000000000000000000000006"
swhid: "swh:1:dir:0000000000000000000000000000000000000008"
swhid: "swh:1:rev:0000000000000000000000000000000000000009"
swhid: "swh:1:dir:0000000000000000000000000000000000000012"
swhid: "swh:1:snp:0000000000000000000000000000000000000022"
swhid: "swh:1:rel:0000000000000000000000000000000000000010"
swhid: "swh:1:snp:0000000000000000000000000000000000000020"
swhid: "swh:1:rev:0000000000000000000000000000000000000013"
swhid: "swh:1:ori:8f50d3f60eae370ddbf85c86219c55108a350165"
swhid: "swh:1:ori:83404f995118bd25774f4ac14422a8f175e7a054"
swhid: "swh:1:rev:0000000000000000000000000000000000000018"
swhid: "swh:1:rel:0000000000000000000000000000000000000021"
swhid: "swh:1:rel:0000000000000000000000000000000000000019"


In [16]:
# starting from content 4 moving up...
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    src0 = "swh:1:cnt:0000000000000000000000000000000000000004"
    response = stub.Traverse(swhgraph.TraversalRequest(
    src=[src0],
    direction=swhgraph.GraphDirection.BACKWARD,
    mask=FieldMask(paths=["swhid"])))
    for item in response:
        print(f'swhid: "{item.swhid}"')

swhid: "swh:1:cnt:0000000000000000000000000000000000000004"
swhid: "swh:1:dir:0000000000000000000000000000000000000006"
swhid: "swh:1:dir:0000000000000000000000000000000000000008"
swhid: "swh:1:rev:0000000000000000000000000000000000000009"
swhid: "swh:1:dir:0000000000000000000000000000000000000012"
swhid: "swh:1:snp:0000000000000000000000000000000000000022"
swhid: "swh:1:rel:0000000000000000000000000000000000000010"
swhid: "swh:1:snp:0000000000000000000000000000000000000020"
swhid: "swh:1:rev:0000000000000000000000000000000000000013"
swhid: "swh:1:ori:8f50d3f60eae370ddbf85c86219c55108a350165"
swhid: "swh:1:ori:83404f995118bd25774f4ac14422a8f175e7a054"
swhid: "swh:1:rev:0000000000000000000000000000000000000018"
swhid: "swh:1:rel:0000000000000000000000000000000000000021"
swhid: "swh:1:rel:0000000000000000000000000000000000000019"


In [17]:
# starting from content 15 moving up...
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    src0 = "swh:1:cnt:0000000000000000000000000000000000000015"
    response = stub.Traverse(swhgraph.TraversalRequest(
    src=[src0],
    direction=swhgraph.GraphDirection.BACKWARD,
    mask=FieldMask(paths=["swhid"])))
    for item in response:
        print(f'swhid: "{item.swhid}"')

swhid: "swh:1:cnt:0000000000000000000000000000000000000015"
swhid: "swh:1:dir:0000000000000000000000000000000000000016"
swhid: "swh:1:dir:0000000000000000000000000000000000000017"
swhid: "swh:1:rev:0000000000000000000000000000000000000018"
swhid: "swh:1:rel:0000000000000000000000000000000000000021"
swhid: "swh:1:rel:0000000000000000000000000000000000000019"
swhid: "swh:1:snp:0000000000000000000000000000000000000022"
swhid: "swh:1:ori:8f50d3f60eae370ddbf85c86219c55108a350165"


In [18]:
# only containing certain edges 
# this will only print revision edges
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.Traverse(swhgraph.TraversalRequest(
    src=["swh:1:rev:0000000000000000000000000000000000000018"],
    edges="rev:rev",
    mask=FieldMask(paths=["swhid"])))
    for item in response:
        print(f'swhid: "{item.swhid}"')

swhid: "swh:1:rev:0000000000000000000000000000000000000018"
swhid: "swh:1:rev:0000000000000000000000000000000000000013"
swhid: "swh:1:rev:0000000000000000000000000000000000000009"
swhid: "swh:1:rev:0000000000000000000000000000000000000003"


In [30]:
# only goes forward and shows as requested, here directories connecting 17 and content connecting these directiories
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    src = "swh:1:dir:0000000000000000000000000000000000000017"
    response = stub.Traverse(swhgraph.TraversalRequest(
    src=[src],
    edges="dir:dir,dir:cnt",
    mask=FieldMask(paths=["swhid"])))
    for item in response:
        print(f'swhid: "{item.swhid}"')

swhid: "swh:1:dir:0000000000000000000000000000000000000017"
swhid: "swh:1:dir:0000000000000000000000000000000000000016"
swhid: "swh:1:cnt:0000000000000000000000000000000000000014"
swhid: "swh:1:cnt:0000000000000000000000000000000000000015"


### finding node

In [32]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.Traverse(swhgraph.TraversalRequest(
    src=["swh:1:dir:0000000000000000000000000000000000000006"],
    return_nodes=swhgraph.NodeFilter(types="ori"),
    direction=swhgraph.GraphDirection.BACKWARD,
    mask=FieldMask(paths=["swhid"])))
    for item in response:
        print(f'swhid: "{item.swhid}"')

swhid: "swh:1:ori:8f50d3f60eae370ddbf85c86219c55108a350165"
swhid: "swh:1:ori:83404f995118bd25774f4ac14422a8f175e7a054"


### Filtering the origin URLs

In [40]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.Traverse(swhgraph.TraversalRequest(
    src=["swh:1:dir:0000000000000000000000000000000000000006"],
    return_nodes=swhgraph.NodeFilter(types="ori"),
    direction=swhgraph.GraphDirection.BACKWARD,
    mask=FieldMask(paths=["swhid", "ori.url"])))
    for item in response:
        print(f'swhid: "{item.swhid}"')
        print(f'url: "{item.ori.url}"')

swhid: "swh:1:ori:8f50d3f60eae370ddbf85c86219c55108a350165"
url: "https://example.com/swh/graph2"
swhid: "swh:1:ori:83404f995118bd25774f4ac14422a8f175e7a054"
url: "https://example.com/swh/graph"


In [41]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.Traverse(swhgraph.TraversalRequest(
    src=["swh:1:dir:0000000000000000000000000000000000000016"],
    return_nodes=swhgraph.NodeFilter(types="ori"),
    direction=swhgraph.GraphDirection.BACKWARD,
    mask=FieldMask(paths=["swhid", "ori.url"])))
    for item in response:
        print(f'swhid: "{item.swhid}"')
        print(f'url: "{item.ori.url}"')

swhid: "swh:1:ori:8f50d3f60eae370ddbf85c86219c55108a350165"
url: "https://example.com/swh/graph2"


In [86]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.Traverse(swhgraph.TraversalRequest(
    src=["swh:1:dir:0000000000000000000000000000000000000012"],
    return_nodes=swhgraph.NodeFilter(types="dir"),
    direction=swhgraph.GraphDirection.FORWARD,
    mask=FieldMask(paths=["swhid"])))
    for item in response:
        print(f'swhid: "{item.swhid}"')

swhid: "swh:1:dir:0000000000000000000000000000000000000012"
swhid: "swh:1:dir:0000000000000000000000000000000000000008"
swhid: "swh:1:dir:0000000000000000000000000000000000000006"


### Path between two node sets

In [87]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.FindPathBetween(swhgraph.FindPathBetweenRequest(
    src=["swh:1:snp:0000000000000000000000000000000000000020"],
    dst=["swh:1:cnt:0000000000000000000000000000000000000004"],
    mask=FieldMask(paths=["swhid"])))
    for item in response.node:
        print(f'swhid: "{item.swhid}"')

swhid: "swh:1:snp:0000000000000000000000000000000000000020"
swhid: "swh:1:rev:0000000000000000000000000000000000000009"
swhid: "swh:1:dir:0000000000000000000000000000000000000008"
swhid: "swh:1:dir:0000000000000000000000000000000000000006"
swhid: "swh:1:cnt:0000000000000000000000000000000000000004"


### shortest path

In [88]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.FindPathBetween(swhgraph.FindPathBetweenRequest(
    src=["swh:1:dir:0000000000000000000000000000000000000006"],
    dst=["swh:1:rel:0000000000000000000000000000000000000019"],
    direction=swhgraph.GraphDirection.BACKWARD,
    mask=FieldMask(paths=["swhid"])))
    for item in response.node:
        print(f'swhid: "{item.swhid}"')

swhid: "swh:1:dir:0000000000000000000000000000000000000006"
swhid: "swh:1:dir:0000000000000000000000000000000000000008"
swhid: "swh:1:rev:0000000000000000000000000000000000000009"
swhid: "swh:1:rev:0000000000000000000000000000000000000013"
swhid: "swh:1:rev:0000000000000000000000000000000000000018"
swhid: "swh:1:rel:0000000000000000000000000000000000000019"


### common anchestor

In [94]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.FindPathBetween(swhgraph.FindPathBetweenRequest(
    src=["swh:1:cnt:0000000000000000000000000000000000000004"],
    dst=["swh:1:cnt:0000000000000000000000000000000000000015"],
    direction=swhgraph.GraphDirection.BACKWARD,
    direction_reverse=swhgraph.GraphDirection.BACKWARD,
    mask=FieldMask(paths=["swhid"]),
))
for item in response.node:
    print(f'swhid: "{item.swhid}"')

swhid: "swh:1:cnt:0000000000000000000000000000000000000004"
swhid: "swh:1:dir:0000000000000000000000000000000000000006"
swhid: "swh:1:dir:0000000000000000000000000000000000000008"
swhid: "swh:1:rev:0000000000000000000000000000000000000009"
swhid: "swh:1:snp:0000000000000000000000000000000000000022"
swhid: "swh:1:rel:0000000000000000000000000000000000000021"
swhid: "swh:1:rev:0000000000000000000000000000000000000018"
swhid: "swh:1:dir:0000000000000000000000000000000000000017"
swhid: "swh:1:dir:0000000000000000000000000000000000000016"
swhid: "swh:1:cnt:0000000000000000000000000000000000000015"


In [95]:
with grpc.insecure_channel(GRAPH_GRPC_SERVER) as channel:
    stub = swhgraph_grpc.TraversalServiceStub(channel)
    response = stub.FindPathBetween(swhgraph.FindPathBetweenRequest(
    src=["swh:1:rel:0000000000000000000000000000000000000010"],
    dst=["swh:1:rel:0000000000000000000000000000000000000021"],
    direction=swhgraph.GraphDirection.FORWARD,
    direction_reverse=swhgraph.GraphDirection.FORWARD,
    mask=FieldMask(paths=["swhid"]),
))
for item in response.node:
    print(f'swhid: "{item.swhid}"')

swhid: "swh:1:rel:0000000000000000000000000000000000000010"
swhid: "swh:1:rev:0000000000000000000000000000000000000009"
swhid: "swh:1:rev:0000000000000000000000000000000000000013"
swhid: "swh:1:rev:0000000000000000000000000000000000000018"
swhid: "swh:1:rel:0000000000000000000000000000000000000021"
